In [ ]:
%pip install python-dotenv

## 环境变量

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

AI_TEMPERATURE=0.0
AI_MAX_TOKENS=2048

## 模型加载

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    embedding = OpenAIEmbeddings(deployment = os.environ.get('DEPLOYMENT_NAME_EMBEDDING'),chunk_size=1)
    
    return llm,chat,embedding

llm,chat,embed = load_azureLLM()

In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent

from langchain.memory import ConversationBufferMemory

In [ ]:

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
from langchain.chains import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm=llm,verbose=True)
tools = [
    Tool(
        name = "计算器",
        func=llm_math_chain.run,
        description="当你需要数学计算时，这个工具很有用"
    ),
]

In [ ]:
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    memory=memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True)

In [ ]:
agent_chain.run(input="hello, i'm linda!")

In [ ]:
agent_chain.run(input="2+5等于多少？")

In [ ]:
agent_chain.run(input="我叫什么？")

In [ ]:
agent_chain.run(input="今天会下雨吗？")

In [ ]:
agent_chain.run(input="2的平方根是多少？")

In [ ]:
agent_chain.run(input="我叫什么？")

In [ ]:
print(memory.buffer)